# PhaseObjC - MELTS Liquid Phase
## Derivative Testers

Required Python code to load the phase library.

In [ ]:
from ctypes import cdll
from ctypes import util
from rubicon.objc import ObjCClass, objc_method
cdll.LoadLibrary(util.find_library('phaseobjc'))
import numpy as np

### Create a Python reference to the `Feldspar` solution phase class, and instantiate an instance of that class.  
LiquidMelts  
LiquidMeltsPlusCO2  
LiquidMeltsPlusOldH2OandNewCO2

In [ ]:
Liquid = ObjCClass('LiquidMeltsPlusOldH2OandNewCO2')
obj = Liquid.alloc().init()

### Obtain properties of the phase inherited from the PhaseBase class.  

In [ ]:
print (obj.phaseName)

## Solution Protocol Functions

### Solution component and species number:

In [ ]:
nc = obj.numberOfSolutionComponents()
print ('Number of components = ', nc)
ns = obj.numberOfSolutionSpecies()
print ('Number of species = ', ns)

### Information about solution components:

In [ ]:
PhaseBase = ObjCClass('PhaseBase')
print ("{0:>20s} {1:>20s} {2:>15s}".format('component name', 'formula', 'MW (g/mol)'))
for i in range(0, nc):
    component = obj.componentAtIndex_(i)
    print ("{0:>20s} {1:>20s} {2:15.3f}".format(component.phaseName, component.phaseFormula, component.mw))

### Create a vector of moles of endmember components.
Allocate a "c"-type pointer to a double precision one-dimensional array, and initialize the array to hold the moles of each component in the solution.  
Composition correcponds to (in oxide wt %): 'SiO2':  77.5, 'TiO2':   0.08, 'Al2O3': 12.5, 'Fe2O3':  0.207,'Cr2O3':  0.0, 'FeO':    0.473, 'MnO':    0.0,'MgO':    0.03, 'NiO':    0.0, 'CoO':    0.0,'CaO':    0.43, 'Na2O':   3.98, 'K2O':    4.88, 'P2O5':   0.0, 'H2O':    5.5,'CO2':    0.0

In [ ]:
import ctypes
m = (ctypes.c_double*nc)()
ctypes.cast(m, ctypes.POINTER(ctypes.c_double))
m[0] = 0.665792
m[1] = 0.000600
m[2] = 0.042436
m[3] = 0.000777
m[4] = 0.0
m[5] = 0.001973
m[6] = 0.0
m[7] = 0.000223
m[8] = 0.0
m[9] = 0.0
m[10] = 0.004596
m[11] = 0.038493
m[12] = 0.062105
m[13] = 0.0
m[14] = 0.183006
m[15] = 0.001
for i in range (0, nc):
    component = obj.componentAtIndex_(i)
    print ('moles of {0:10s} = {1:10.6f}'.format(component.phaseName, m[i]))

# Derivative testers 

In [ ]:
from scipy.misc import derivative
t = 760+273.15
p = 1750

In [ ]:
dT = 1
dP = 2
dm = 3
def test_h(t,p):
    h_est = obj.getGibbsFreeEnergyFromMolesOfComponents_andT_andP_(m,t,p) \
          + t*obj.getEntropyFromMolesOfComponents_andT_andP_(m, t, p)
    h_act = obj.getEnthalpyFromMolesOfComponents_andT_andP_(m, t, p)
    h_err = (h_est-h_act)*100.0/h_act
    print ("H       {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(h_err, h_est, h_act))
def g(x, dx=dT, n=0):
    if dx == dT:
        return obj.getGibbsFreeEnergyFromMolesOfComponents_andT_andP_(m, x, p)
    elif dx == dP:
        return obj.getGibbsFreeEnergyFromMolesOfComponents_andT_andP_(m, t, x)
    elif dx == dm:
        if n < 0 or n >= nc:
            return 0.0
        mold = m[n]
        m[n] = x
        result = obj.getGibbsFreeEnergyFromMolesOfComponents_andT_andP_(m, t, p)
        m[n] = mold
        return result
def test_g_dt(t,p):
    s_est = -derivative(g, t, args=(dT,))
    s_act = obj.getEntropyFromMolesOfComponents_andT_andP_(m, t, p)
    s_err = (s_est-s_act)*100.0/s_act
    print ("S       {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(s_err, s_est, s_act))
def test_a_mu(t,p):
    dgdm = obj.getDgDmFromMolesOfComponents_andT_andP_(m, t, p)
    activity = obj.getActivityFromMolesOfComponents_andT_andP_(m, t, p)
    potential = obj.getChemicalPotentialFromMolesOfComponents_andT_andP_(m, t, p)
    for i in range(0,nc):
        if m[i] != 0.0:
            da_est = dgdm.valueAtIndex_(i)
            da_act = 8.3143*t*np.log(activity.valueAtIndex_(i)) \
                   + obj.componentAtIndex_(i).getGibbsFreeEnergyFromT_andP_(t, p)
            da_err = (da_est-da_act)*100.0/da_act
            print ("RT ln a {0:>10s} {1:10.6f} % error, est: {2:15.6e} act: {3:15.6e}".format( \
                obj.componentAtIndex_(i).phaseName, da_err, da_est, da_act))
    for i in range(0,nc):
        if m[i] != 0.0:
            dmu_est = dgdm.valueAtIndex_(i)
            dmu_act = potential.valueAtIndex_(i)
            dmu_err = (dmu_est-dmu_act)*100.0/dmu_act
            print ("mu      {0:>10s} {1:10.6f} % error, est: {2:15.6e} act: {3:15.6e}".format( \
                obj.componentAtIndex_(i).phaseName, dmu_err, dmu_est, dmu_act))
def test_g_dp(t,p):
    v_est = derivative(g, p, args=(dP,))
    v_act = obj.getVolumeFromMolesOfComponents_andT_andP_(m, t, p)
    v_err = (v_est-v_act)*100.0/v_act
    print ("V       {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(v_err, v_est, v_act))
def test_dg_dm(t,p):
    dgdm = obj.getDgDmFromMolesOfComponents_andT_andP_(m, t, p)
    for i in range (0,nc):
        if m[i] != 0.0:
            dm_est = derivative(g, m[i], args=(dm,i,), dx=1e-4)
            dm_act = dgdm.valueAtIndex_(i)
            dm_err = (dm_est-dm_act)*100.0/dm_act
            print ("dgdm    {0:>10s} {1:10.6f} % error, est: {2:15.6e} act: {3:15.6e}".format( \
                obj.componentAtIndex_(i).phaseName, dm_err, dm_est, dm_act))
def dgdm(x, i=0, j=0):
    if i < 0 or i >= nc:
        return 0.0
    if j < 0 or j >= nc:
        return 0.0
    mold = m[j]
    m[j] = x
    dgdm = obj.getDgDmFromMolesOfComponents_andT_andP_(m, t, p)
    m[j] = mold
    return dgdm.valueAtIndex_(i)
def test_d2g_dm2(t,p):
    d2gdm2 = obj.getD2gDm2FromMolesOfComponents_andT_andP_(m, t, p)
    for i in range (0,nc):
        if m[i] != 0.0:
            for j in range (0,nc):
                if m[j] != 0.0:
                    d2m_est = derivative(dgdm, m[j], args=(i,j,), dx=1e-4)
                    d2m_act = d2gdm2.valueAtRowIndex_andColIndex_(i, j)
                    d2m_err = (d2m_est-d2m_act)*100.0/d2m_act
                    print ("d2gdm2 {0:>10s} {1:>10s} {2:10.6f} % error, est: {3:15.6e} act: {4:15.6e}".format( \
                    obj.componentAtIndex_(i).phaseName, obj.componentAtIndex_(j).phaseName, d2m_err, d2m_est, d2m_act))
def activity(x, i=0, j=0):
    if i < 0 or i >= nc:
        return 0.0
    if j < 0 or j >= nc:
        return 0.0
    mold = m[j]
    m[j] = x
    a = obj.getActivityFromMolesOfComponents_andT_andP_(m, t, p)
    m[j] = mold
    return a.valueAtIndex_(i)
def test_da_dm(t,p):
    dadm = obj.getDaDmFromMolesOfComponents_andT_andP_(m, t, p)
    for i in range (0,nc):
        if m[i] != 0.0:
            for j in range (0,nc):
                if m[j] != 0.0:
                    dadm_est = derivative(activity, m[j], args=(i,j,), dx=1e-6)
                    dadm_act = dadm.valueAtRowIndex_andColIndex_(i, j)
                    dadm_err = (dadm_est-dadm_act)*100.0/dadm_act
                    print ("dadm {0:>10s} {1:>10s} {2:10.6f} % error, est: {3:15.6e} act: {4:15.6e}".format( \
                    obj.componentAtIndex_(i).phaseName, obj.componentAtIndex_(j).phaseName, \
                    dadm_err, dadm_est, dadm_act))
def s(x, dx=dT, n=0):
    if dx == dT:
        return obj.getEntropyFromMolesOfComponents_andT_andP_(m, x, p)
    elif dx == dP:
        return obj.getEntropyFromMolesOfComponents_andT_andP_(m, t, x)
    elif dx == dm:
        if n < 0 or n >= nc:
            return 0.0
        mold = m[n]
        m[n] = x
        result = obj.getEntropyFromMolesOfComponents_andT_andP_(m, t, p)
        m[n] = mold
        return result
def test_s_dt(t,p):
    cp_est = t*derivative(s, t, args=(dT,))
    cp_act = obj.getHeatCapacityFromMolesOfComponents_andT_andP_(m, t, p)
    cp_err = (cp_est-cp_act)*100.0/cp_act
    print ("Cp      {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(cp_err, cp_est, cp_act))
def test_ds_dm(t,p):
    dsdm = obj.getDsDmFromMolesOfComponents_andT_andP_(m, t, p)
    for i in range (0,nc):
        if m[i] != 0.0:
            dm_est = derivative(s, m[i], args=(dm,i,), dx=1e-4)
            dm_act = dsdm.valueAtIndex_(i)
            dm_err = (dm_est-dm_act)*100.0/dm_act
            print ("dsdm    {0:>10s} {1:10.6f} % error, est: {2:15.6e} act: {3:15.6e}".format( \
                obj.componentAtIndex_(i).phaseName, dm_err, dm_est, dm_act))
def dsdm(x, i=0, j=0):
    if i < 0 or i >= nc:
        return 0.0
    if j < 0 or j >= nc:
        return 0.0
    mold = m[j]
    m[j] = x
    dsdm = obj.getDsDmFromMolesOfComponents_andT_andP_(m, t, p)
    m[j] = mold
    return dsdm.valueAtIndex_(i)
def test_d2s_dm2(t,p):
    d2sdm2 = obj.getD2sDm2FromMolesOfComponents_andT_andP_(m, t, p)
    for i in range (0,nc):
        if m[i] != 0.0:
            for j in range (0,nc):
                if m[j] != 0.0:
                    d2m_est = derivative(dsdm, m[j], args=(i,j,), dx=1e-4)
                    d2m_act = d2sdm2.valueAtRowIndex_andColIndex_(i, j)
                    d2m_err = (d2m_est-d2m_act)*100.0/d2m_act
                    print ("d2sdm2 {0:>10s} {1:>10s} {2:10.6f} % error, est: {3:15.6e} act: {4:15.6e}".format( \
                    obj.componentAtIndex_(i).phaseName, obj.componentAtIndex_(j).phaseName, d2m_err, d2m_est, d2m_act))
def cp(x, dx=dT, n=0):
    if dx == dT:
        return obj.getHeatCapacityFromMolesOfComponents_andT_andP_(m, x, p)
    elif dx == dP:
        return obj.getHeatCapacityFromMolesOfComponents_andT_andP_(m, t, x)
    elif dx == dm:
        if n < 0 or n >= nc:
            return 0.0
        mold = m[n]
        m[n] = x
        result = obj.getHeatCapacityFromMolesOfComponents_andT_andP_(m, t, p)
        m[n] = mold
        return result
def test_cp_dt(t,p):
    dcpdt_est = derivative(cp, t, args=(True,), dx=1e-4)
    dcpdt_act = obj.getDcpDtFromMolesOfComponents_andT_andP_(m, t, p)
    dcpdt_err = (dcpdt_est-dcpdt_act)*100.0/dcpdt_act
    print ("dCpDt   {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(dcpdt_err, dcpdt_est, dcpdt_act))
def test_dcp_dm(t,p):
    dcpdm = obj.getDCpDmFromMolesOfComponents_andT_andP_(m, t, p)
    for i in range (0,nc):
        if m[i] != 0.0:
            dm_est = derivative(cp, m[i], args=(dm,i,), dx=1e-4)
            dm_act = dcpdm.valueAtIndex_(i)
            dm_err = (dm_est-dm_act)*100.0/dm_act
            print ("dcpdm   {0:>10s} {1:10.6f} % error, est: {2:15.6e} act: {3:15.6e}".format( \
                obj.componentAtIndex_(i).phaseName, dm_err, dm_est, dm_act))
def v(x, dx=dT, n=0):
    if dx == dT:
        return obj.getVolumeFromMolesOfComponents_andT_andP_(m, x, p)
    elif dx == dP:
        return obj.getVolumeFromMolesOfComponents_andT_andP_(m, t, x)
    elif dx == dm:
        if n < 0 or n >= nc:
            return 0.0
        mold = m[n]
        m[n] = x
        result = obj.getVolumeFromMolesOfComponents_andT_andP_(m, t, p)
        m[n] = mold
        return result
def test_v_dt(t,p):
    dvdt_est = derivative(v, t, args=(dT,))
    dvdt_act = obj.getDvDtFromMolesOfComponents_andT_andP_(m, t, p)
    dvdt_err = (dvdt_est-dvdt_act)*100.0/dvdt_act
    print ("dVdT    {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(dvdt_err, dvdt_est, dvdt_act))
def test_v_dp(t,p):
    dvdp_est = derivative(v, p, args=(dP,))
    dvdp_act = obj.getDvDpFromMolesOfComponents_andT_andP_(m, t, p)
    dvdp_err = (dvdp_est-dvdp_act)*100.0/dvdp_act
    print ("dVdP    {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(dvdp_err, dvdp_est, dvdp_act))
def test_dv_dm(t,p):
    dvdm = obj.getDvDmFromMolesOfComponents_andT_andP_(m, t, p)
    for i in range (0,nc):
        if m[i] != 0.0:
            dm_est = derivative(v, m[i], args=(dm,i,), dx=1e-4)
            dm_act = dvdm.valueAtIndex_(i)
            dm_err = (dm_est-dm_act)*100.0/dm_act
            print ("dvdm    {0:>10s} {1:10.6f} % error, est: {2:15.6e} act: {3:15.6e}".format( \
                obj.componentAtIndex_(i).phaseName, dm_err, dm_est, dm_act))
def dvdm(x, i=0, j=0):
    if i < 0 or i >= nc:
        return 0.0
    if j < 0 or j >= nc:
        return 0.0
    mold = m[j]
    m[j] = x
    dvdm = obj.getDvDmFromMolesOfComponents_andT_andP_(m, t, p)
    m[j] = mold
    return dvdm.valueAtIndex_(i)
def test_d2v_dm2(t,p):
    d2vdm2 = obj.getD2vDm2FromMolesOfComponents_andT_andP_(m, t, p)
    for i in range (0,nc):
        if m[i] != 0.0:
            for j in range (0,nc):
                if m[j] != 0.0:
                    d2m_est = derivative(dvdm, m[j], args=(i,j,), dx=1e-4)
                    d2m_act = d2vdm2.valueAtRowIndex_andColIndex_(i, j)
                    d2m_err = (d2m_est-d2m_act)*100.0/d2m_act
                    print ("d2vdm2 {0:>10s} {1:>10s} {2:10.6f} % error, est: {3:15.6e} act: {4:15.6e}".format( \
                    obj.componentAtIndex_(i).phaseName, obj.componentAtIndex_(j).phaseName, d2m_err, d2m_est, d2m_act))
def dvdt(x, dx=dT, n=0):
    if dx == dT:
        return obj.getDvDtFromMolesOfComponents_andT_andP_(m, x, p)
    elif dx == dP:
        return obj.getDvDtFromMolesOfComponents_andT_andP_(m, t, x)
    elif dx == dm:
        if n < 0 or n >= nc:
            return 0.0
        mold = m[n]
        m[n] = x
        result = obj.getDvDtFromMolesOfComponents_andT_andP_(m, t, p)
        m[n] = mold
        return result
def dvdp(x, dx=dT, n=0):
    if dx == dT:
        return obj.getDvDpFromMolesOfComponents_andT_andP_(m, x, p)
    elif dx == dP:
        return obj.getDvDpFromMolesOfComponents_andT_andP_(m, t, x)
    elif dx == dm:
        if n < 0 or n >= nc:
            return 0.0
        mold = m[n]
        m[n] = x
        result = obj.getDvDpFromMolesOfComponents_andT_andP_(m, t, p)
        m[n] = mold
        return result
def test_dvdt_dt(t,p):
    d2vdt2_est = derivative(dvdt, t, args=(dT,), dx=1e-4)
    d2vdt2_act = obj.getD2vDt2FromMolesOfComponents_andT_andP_(m, t, p)
    d2vdt2_err = (d2vdt2_est-d2vdt2_act)*100.0/d2vdt2_act
    print ("d2VdT2  {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(d2vdt2_err, d2vdt2_est, d2vdt2_act))
def test_dvdt_dp(t,p):
    d2vdtdp_est = derivative(dvdt, p, args=(dP,), dx=1e-4)
    d2vdtdp_act = obj.getD2vDtDpFromMolesOfComponents_andT_andP_(m, t, p)
    d2vdtdp_err = (d2vdtdp_est-d2vdtdp_act)*100.0/d2vdtdp_act
    print ("d2VdTdP {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(d2vdtdp_err, d2vdtdp_est, d2vdtdp_act))

def test_dvdp_dt(t,p):
    d2vdtdp_est = derivative(dvdp, t, args=(dT,), dx=1e-4)
    d2vdtdp_act = obj.getD2vDtDpFromMolesOfComponents_andT_andP_(m, t, p)
    d2vdtdp_err = (d2vdtdp_est-d2vdtdp_act)*100.0/d2vdtdp_act
    print ("d2VdTDp {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(d2vdtdp_err, d2vdtdp_est, d2vdtdp_act))
def test_dvdp_dp(t,p):
    d2vdp2_est = derivative(dvdp, p, args=(dP,), dx=1e-4)
    d2vdp2_act = obj.getD2vDp2FromMolesOfComponents_andT_andP_(m, t, p)
    d2vdp2_err = (d2vdp2_est-d2vdp2_act)*100.0/d2vdp2_act
    print ("d2VdP2  {0:10.6f} % error, est: {1:15.6e} act: {2:15.6e}".format(d2vdp2_err, d2vdp2_est, d2vdp2_act))
def test_d2v_dtdm(t,p):
    d2vdmdt = obj.getD2vDmDtFromMolesOfComponents_andT_andP_(m, t, p)
    for i in range (0,nc):
        if m[i] != 0.0:
            dm_est = derivative(dvdt, m[i], args=(dm,i,), dx=1e-4)
            dm_act = d2vdmdt.valueAtIndex_(i)
            dm_err = (dm_est-dm_act)*100.0/dm_act
            print ("d2vdmdt {0:>10s} {1:10.6f} % error, est: {2:15.6e} act: {3:15.6e}".format( \
                obj.componentAtIndex_(i).phaseName, dm_err, dm_est, dm_act))
def test_d2v_dpdm(t,p):
    d2vdmdp = obj.getD2vDmDpFromMolesOfComponents_andT_andP_(m, t, p)
    for i in range (0,nc):
        if m[i] != 0.0:
            dm_est = derivative(dvdp, m[i], args=(dm,i,), dx=1e-4)
            dm_act = d2vdmdp.valueAtIndex_(i)
            dm_err = (dm_est-dm_act)*100.0/dm_act
            print ("d2vdmdp {0:>10s} {1:10.6f} % error, est: {2:15.6e} act: {3:15.6e}".format( \
                obj.componentAtIndex_(i).phaseName, dm_err, dm_est, dm_act))

In [ ]:
print(obj.getGibbsFreeEnergyFromMolesOfComponents_andT_andP_(m,t,p))

In [ ]:
test_h(t,p)
test_g_dt(t,p)
test_s_dt(t,p)
test_cp_dt(t,p)
test_g_dp(t,p)
test_v_dt(t,p)
test_v_dp(t,p)
test_dvdt_dt(t,p)
test_dvdt_dp(t,p)
test_dvdp_dt(t,p)
test_dvdp_dp(t,p)

### Compositional derivatives

In [ ]:
test_dg_dm(t,p)
test_ds_dm(t,p)
test_dv_dm(t,p)
test_dcp_dm(t,p)
test_d2v_dtdm(t,p)
test_d2v_dpdm(t,p)

In [ ]:
test_d2g_dm2(t,p)

In [ ]:
test_d2s_dm2(t,p)

In [ ]:
test_d2v_dm2(t,p)

In [ ]:
test_a_mu(t,p)

In [ ]:
test_da_dm(t,p)

### Check integrity of derivatives for endmember component properties

In [ ]:
for j in range (0, nc):
    for i in range (0, nc):
        m[i] = 0.000001
    m[j] = 1.0
    print (j, obj.componentAtIndex_(j).phaseName)
    test_h(t,p)
    test_g_dt(t,p)
    test_s_dt(t,p)
    test_cp_dt(t,p)
    test_g_dp(t,p)
    test_v_dt(t,p)
    test_v_dp(t,p)
    test_dvdt_dt(t,p)
    test_dvdt_dp(t,p)
    test_dvdp_dt(t,p)
    test_dvdp_dp(t,p)